# Custom Evaluation Criteria

In this example, we will see how to create a custom evaluation.

We will create an eval that evaluates whether a user query is expanded into sub-queries, covering all the angles of the original query to retrieve all the necessary information.

# Creating the metric

`flow-eval` makes it easy to create custom evaluations.

In [1]:
from flow_eval.lm import LMEval, RubricItem
from IPython.display import Markdown, display

# Define the criteria
evaluation_criteria = """Do the generated sub-queries provide sufficient breadth to cover all aspects of the main query?"""

# Define the rubric using RubricItem's
rubric = [
    RubricItem(
        score=1,
        description="The sub-queries lack breadth and fail to address multiple important aspects of the main query. They are either too narrow, focusing on only one or two dimensions of the question, or they diverge significantly from the main query's intent. Using these sub-queries alone would result in a severely limited exploration of the topic."),
    RubricItem(
        score=2,
        description="The sub-queries cover some aspects of the main query but lack comprehensive breadth. While they touch on several dimensions of the question, there are still noticeable gaps in coverage. Some important facets of the main query are either underrepresented or missing entirely. Answering these sub-queries would provide a partial, but not complete, exploration of the main topic."),
    RubricItem(
        score=3,
        description="The sub-queries demonstrate excellent breadth, effectively covering all major aspects of the main query. They break down the main question into a diverse set of dimensions, ensuring a comprehensive exploration of the topic. Each significant facet of the main query is represented in the sub-queries, allowing for a thorough and well-rounded investigation of the subject matter."),
]

# We need to define the required inputs and output for the metric
required_inputs = ["query"]
required_output = "sub_queries"

# Create the metric
sub_query_coverage = LMEval(
    name="sub-query-coverage",
    criteria=evaluation_criteria,
    rubric=rubric,
    input_columns=required_inputs,
    output_column=required_output
)

INFO:datasets:PyTorch version 2.4.0 available.


As you can see, we quickly created a custom eval that will instruct the model to evaluate according to the criteria and rubric we set.

In [2]:
sub_query_coverage

LMEval(name='sub-query-coverage', input_columns=['query'], output_column='sub_queries', expected_output_column=None, criteria='Do the generated sub-queries provide sufficient breadth to cover all aspects of the main query?', rubric=[RubricItem(score=1, description="The sub-queries lack breadth and fail to address multiple important aspects of the main query. They are either too narrow, focusing on only one or two dimensions of the question, or they diverge significantly from the main query's intent. Using these sub-queries alone would result in a severely limited exploration of the topic."), RubricItem(score=2, description='The sub-queries cover some aspects of the main query but lack comprehensive breadth. While they touch on several dimensions of the question, there are still noticeable gaps in coverage. Some important facets of the main query are either underrepresented or missing entirely. Answering these sub-queries would provide a partial, but not complete, exploration of the mai

# Running evaluations with the custom metric

Now, we just need to initialize the judge with our custom metric and run the evaluations.

In [3]:
from flow_eval import LMEvaluator
from flow_eval.core import EvalInput
from flow_eval.lm.models import Vllm #, Llamafile, Hf
# from flow_eval import Baseten

# If you are running on an Ampere GPU or newer, create a model using VLLM
model = Vllm()

# Or if not running on Ampere GPU or newer, create a model using no flash attn and Hugging Face Transformers
# model = Hf(flash_attn=False)

# Or create a model using Llamafile if not running an Nvidia GPU & running a Silicon MacOS for example
# model = Llamafile()

# Or create a model using Baseten if you don't want to run locally.
# As a pre-requisite step:
#  - Sign up to Baseten
#  - Generate an api key https://app.baseten.co/settings/api_keys
#  - Set the api key as an environment variable & initialize:
# import os
# os.environ["BASETEN_API_KEY"] = "your_api_key"
# model = Baseten()

# Initialize the evaluator
evaluator = LMEvaluator(eval=sub_query_coverage, model=model)

INFO 01-23 19:32:06 awq_marlin.py:90] The model is convertible to awq_marlin during runtime. Using awq_marlin kernel.
WARNING 01-23 19:32:06 config.py:389] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 01-23 19:32:06 llm_engine.py:226] Initializing an LLM engine (v0.6.1.dev238+ge2c6e0a82) with config: model='flowaicom/Flow-Judge-v0.1-AWQ', speculative_config=None, tokenizer='flowaicom/Flow-Judge-v0.1-AWQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq_marlin, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(gu

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 01-23 19:32:08 model_runner.py:1025] Loading model weights took 2.1717 GB
INFO 01-23 19:32:10 gpu_executor.py:122] # GPU blocks: 3083, # CPU blocks: 682


In [4]:
# Sample to evaluate

query = "I placed an order for a custom-built gaming PC (order #AC-789012) two weeks ago, but the estimated delivery date has changed twice since then. Originally it was supposed to arrive yesterday, then it got pushed to next week, and now the tracking page shows 'Status: Processing' with no delivery estimate at all. I've tried calling customer service, but after waiting on hold for an hour, I was told to check the website. Can you please look into this and explain what's causing the delays, when I can realistically expect my order to arrive, and whether I'm eligible for any kind of compensation or expedited shipping given these repeated delays? I'm especially concerned because I need this computer for an upcoming gaming tournament I'm participating in next month."
sub_queries = "What is the current shipping status of my order? How can I build a PC?"

# bad expansion
eval_input = EvalInput(
    inputs=[
        {"query": query}
    ],
    output={"sub_queries": sub_queries}
)

# Run the evaluation
result = evaluator.evaluate(eval_input)

Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.80s/it, est. speed input: 145.58 toks/s, output: 46.03 toks/s]


In [5]:
# Display the result
display(Markdown(f"__Feedback:__\n{result.feedback}\n\n__Score:__\n{result.score}"))

__Feedback:__
The generated sub-queries fail to provide sufficient breadth to cover all aspects of the main query. The main query addresses multiple issues: repeated delivery date changes, a current shipping status, potential compensation, and expedited shipping options. However, the sub-queries only cover two aspects: the current shipping status and building a PC.

The first sub-query, "What is the current shipping status of my order?" is relevant but only addresses part of the main query's concerns. It doesn't touch on the repeated delivery date changes, compensation, or expedited shipping options.

The second sub-query, "How can I build a PC?" is entirely unrelated to the main query's concerns. It diverges significantly from the main query's intent and doesn't address any of the customer's issues or questions.

Using these sub-queries alone would result in a severely limited exploration of the topic. They fail to address the repeated delivery delays, potential compensation, or expedited shipping options, which were key concerns in the main query. The sub-queries also don't address the urgency of the situation due to the upcoming gaming tournament.

In summary, the sub-queries lack breadth and fail to address multiple important aspects of the main query, resulting in a score of 1.

__Score:__
1

In [6]:
# Good expansion

sub_queries = """1. What is the current status of order #AC-789012, and why has it changed from the original estimated delivery date?
2. What specific factors are causing the delays in processing and shipping this custom-built gaming PC?
3. Based on the current situation, when can the customer realistically expect the order to arrive?
4. Given the repeated delays and changes in estimated delivery, what compensation options (if any) are available to the customer?
5. Is expedited shipping an option at this point, and if so, how would it affect the delivery timeline?
6. How can the urgency of this order be communicated and prioritized, considering the customer's upcoming gaming tournament next month?
7. What steps has customer service already taken to address this issue, and what additional actions can be taken to resolve it?
8. How can the customer receive more frequent and accurate updates about their order status going forward?"""

eval_input = EvalInput(
    inputs=[
        {"query": query}
    ],
    output={"sub_queries": sub_queries}
)

# Run the evaluation
result = evaluator.evaluate(eval_input)

Processed prompts: 100%|██████████| 1/1 [00:07<00:00,  7.22s/it, est. speed input: 163.72 toks/s, output: 46.26 toks/s]


In [7]:
# Display the result
display(Markdown(f"__Feedback:__\n{result.feedback}\n\n__Score:__\n{result.score}"))

__Feedback:__
The generated sub-queries demonstrate excellent breadth and effectively cover all major aspects of the main query. Each significant facet of the main query is represented in the sub-queries, ensuring a comprehensive exploration of the topic.

The sub-queries address the following key points from the main query:

1. Current status and reasons for delivery date changes
2. Specific factors causing delays
3. Realistic delivery expectations
4. Compensation options due to repeated delays
5. Availability of expedited shipping
6. Communication of order urgency
7. Actions taken by customer service and additional steps
8. Improved communication for future updates

These sub-queries break down the main question into diverse dimensions, ensuring a thorough investigation of the subject matter. They cover the technical aspects of the order, the customer service experience, potential solutions, and future preventative measures. This comprehensive approach allows for a well-rounded exploration of the issue at hand.

The sub-queries are well-structured and cover all critical aspects mentioned in the main query, including the customer's urgency due to the upcoming gaming tournament. They address both immediate concerns and long-term solutions, ensuring a complete exploration of the topic.

Overall, the sub-queries provide a thorough and comprehensive framework for investigating and addressing the customer's concerns about the delayed gaming PC order.

__Score:__
3

In [8]:
# Okish expansion

sub_queries = """1. What is the current status of order #AC-7892?
Why has the delivery date changed multiple times?
When will the custom-built gaming PC actually arrive?
What are your store hours on weekends?
How can I get better customer service support?
8. Can the order be expedited?
9. What is the name of the person that is responsible for the company?
"""

eval_input = EvalInput(
    inputs=[
        {"query": query}
    ],
    output={"sub_queries": sub_queries}
)

# Run the evaluation
result = evaluator.evaluate(eval_input)
# Display the result
display(Markdown(f"__Feedback:__\n{result.feedback}\n\n__Score:__\n{result.score}"))

Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.98s/it, est. speed input: 151.78 toks/s, output: 46.54 toks/s]


__Feedback:__
The generated sub-queries demonstrate a good effort to cover various aspects of the main query, but they fall short of providing comprehensive breadth. 

The sub-queries address several key points from the main query:
1. The current status of the order (sub-query 1)
2. The reason for delivery date changes (sub-query 2)
3. The expected arrival time of the order (sub-query 3)
4. The possibility of expediting the order (sub-query 8)

These sub-queries touch on important aspects of the customer's concerns. However, there are noticeable gaps in coverage:
1. The sub-queries do not address the repeated delays mentioned in the main query.
2. They do not specifically address the customer's eligibility for compensation or expedited shipping due to the repeated delays.
3. The sub-queries do not address the customer's need for the PC for an upcoming gaming tournament.

While the sub-queries cover some important facets of the main query, they miss several critical aspects that were highlighted in the original question. This results in a partial exploration of the topic, rather than a comprehensive one.

Therefore, the sub-queries demonstrate some breadth but lack the comprehensive coverage needed to fully address all aspects of the main query.

__Score:__
2